In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

train = pd.read_csv("train.csv")
train["date_time"] = pd.to_datetime(train["date_time"])
train["dayofweek"] = train["date_time"].dt.dayofweek
train["month"] = train["date_time"].dt.month
train["temp_diff"] = train["high_temp"] - train["low_temp"]

y = train["number_of_rentals"]
X = train.drop(columns=["date_time", "number_of_rentals"])
categorical_cols = ["wind_direction", "sky_condition", "precipitation_form", "hour", "dayofweek", "month"]
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.05, random_state=42)
model.fit(X_scaled, np.log1p(y))

test = pd.read_csv("test.csv")
test["date_time"] = pd.to_datetime(test["date_time"])
test["dayofweek"] = test["date_time"].dt.dayofweek
test["month"] = test["date_time"].dt.month
test["temp_diff"] = test["high_temp"] - test["low_temp"]

test_features = test.drop(columns=["date_time"])
test_features = pd.get_dummies(test_features, columns=categorical_cols, drop_first=True)

for col in X.columns:
    if col not in test_features.columns:
        test_features[col] = 0
test_features = test_features[X.columns]

test_scaled = scaler.transform(test_features)
pred_log = model.predict(test_scaled)
pred = np.expm1(pred_log).astype(int)

sample = pd.read_csv("sample_submission.csv")
sample["number_of_rentals"] = pred
sample.to_csv('submission_xgboost.csv', index=False)

C:\Users\김은교\AppData\Local\Temp\ipykernel_19208\2354348062.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_features[col] = 0
C:\Users\김은교\AppData\Local\Temp\ipykernel_19208\2354348062.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_features[col] = 0
C:\Users\김은교\AppData\Local\Temp\ipykernel_19208\2354348062.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca